## 日本語BERTでlivedoorニュースを教師あり学習で分類

In [1]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [2]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

## 準備1：Livedoorニュースをダウンロードしてtsvファイル化

参考：https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb


In [3]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2020-05-19 08:11:22--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.3’

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.87MB/s    in 5.2s    

2020-05-19 08:11:28 (1.64 MB/s) - ‘ldcc-20140209.tar.gz.3’ saved [8855190/8855190]



In [4]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['dokujo-tsushin', 'README.txt', 'it-life-hack', 'CHANGES.txt', 'smax', 'sports-watch', 'kaden-channel', 'movie-enter', 'topic-news', 'livedoor-homme', 'peachy']
カテゴリー数: 9
['dokujo-tsushin', 'it-life-hack', 'smax', 'sports-watch', 'kaden-channel', 'movie-enter', 'topic-news', 'livedoor-homme', 'peachy']


In [5]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある


0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



In [0]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [0]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [8]:
# 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])


婚活中の綾さん(29歳)の理想の結婚相手は転勤のない職種で実家の近くに住んでくれる人。綾さんは二人姉妹の姉。できれば自分の親と同居してくれるマスオさんが理想だそうだ。独女の親との同居率は年々増加している。親と同居していれば、ひとり暮らしの独女より話す頻度も多く繋がりも深くなり、その親密な関係は結婚してもなお続いていくようだ。「毎日実家で夕食を食べています。ママは料理がうまいし、夫もママの手料理を楽しみにしているから、ママも張り切ってます」結婚3年目のマナさん(33歳)は、実家に食費として月に2万円をいれているだけ。家で料理を作らないと光熱費も浮くし、貯金もできる。いずれ実家を二世帯住宅に建て替え、親との同居も考えているそうだ。「休日の度に車で30分の距離のマンションに住む娘が婿と一緒にくる」というのは32歳の既婚の娘を持つ良子さん(57歳)。「家にある食料やもらいものの器や洗剤まで根こそぎ持っていきますね。まるで泥棒が来たみたいです」と苦笑する。「主人は、“お前も嫁に行ったんだから、いつまでも実家を頼ってはいけない”と建て前で叱っていますが、内心は娘の顔が見られるのが嬉しいようです」娘さんは現在妊娠5カ月。出産前後は実家で暮らすそうだ。「今の時代、専業主婦は夢のまた夢。若い世代の収入の減少で、女性は結婚しても働かなければなりません。会社から疲れて帰ってきて、食事の支度をして休日には掃除や洗濯をするなんて、結婚生活は女性の負担が大きいように思います。ましてや女性には出産がありますから、実家の母の手助けは必須だと思いますよ」と良子さん。3歳の子供がいるヒロさん(35歳)も、実家の母がいなければ働きながらの子育ては難しいという。「保育園の緊急連絡先ですが、保育園に一番早く駆けつけることができるのが実家の母なので、1番が実家の母、2番が私。3番目が夫になっています。夫は出張も多くまったく役に立たないので母が頼りです。急なお迎えって滅多にないと思っていたんですけど、元気がなかったり、ちょっとぐったりしていても、すぐに連絡がきます。37度5分でもう緊急連絡です。母が迎えに行って、その足で病院にも連れて行ってくれるので本当に助かります」お子さんが1番好きなのはおばあちゃま。次がママで3番目がパパ。子供の好きな順位が緊急連絡先と同じだとヒロさんは笑うが、幼い子供にとって一番頼りにな

In [9]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()


(7376, 2)


,text,label
0,婚活中の綾さん(29歳)の理想の結婚相手は転勤のない職種で実家の近くに住んでくれる人。綾さん...,dokujo-tsushin
1,佐々木倫子といえば80年代に『動物のお医者さん』を大ヒットさせたベテラン漫画家。獣医を目指す...,dokujo-tsushin
2,猛暑続きの今年の夏。7月末、ストッキングの生産数も10年間で4分の1になっているというニュー...,dokujo-tsushin
3,監督失格という“タイトルに惹かれて”、マスコミ試写に出かけた。映画『監督失格』は、女優・林由...,dokujo-tsushin
4,SNSで学生時代の旧友と再会することも当たり前になってきた。なかには、密かに片思いをしていた...,dokujo-tsushin


In [10]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()


{0: 'dokujo-tsushin', 1: 'it-life-hack', 2: 'smax', 3: 'sports-watch', 4: 'kaden-channel', 5: 'movie-enter', 6: 'topic-news', 7: 'livedoor-homme', 8: 'peachy'}
{'dokujo-tsushin': 0, 'it-life-hack': 1, 'smax': 2, 'sports-watch': 3, 'kaden-channel': 4, 'movie-enter': 5, 'topic-news': 6, 'livedoor-homme': 7, 'peachy': 8}


,text,label_index
0,婚活中の綾さん(29歳)の理想の結婚相手は転勤のない職種で実家の近くに住んでくれる人。綾さん...,0
1,佐々木倫子といえば80年代に『動物のお医者さん』を大ヒットさせたベテラン漫画家。獣医を目指す...,0
2,猛暑続きの今年の夏。7月末、ストッキングの生産数も10年間で4分の1になっているというニュー...,0
3,監督失格という“タイトルに惹かれて”、マスコミ試写に出かけた。映画『監督失格』は、女優・林由...,0
4,SNSで学生時代の旧友と再会することも当たり前になってきた。なかには、密かに片思いをしていた...,0


In [11]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,『アヒルと鴨のコインロッカー』（2007年）や『ゴールデンスランバー』（2010年）など、こ...,5
1,「銀のさら」のWeb限定CMが、「差別的な内容だ」と27日からツイッターなどで物議を醸してい...,6
2,TBS「S1」（1日放送分）には、野球クラブチーム・茨城ゴールデンゴールズの片岡安祐美が出演...,3
3,『容疑者Xの献身』『白夜行』『麒麟の翼』など原作、映画ともに大ヒットを記録している国民的作家...,5
4,多くの人と出会うことができるSNS。そのため、恋人作りのために利用している人も多いのではない...,4


In [12]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)


(1475, 2)
(5901, 2)


In [0]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")


## 準備2：LivedoorニュースをBERT用のDataLoaderにする

Hugginfaceのリポジトリの案内とは異なり、torchtextを使用した手法で実装

In [14]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.7)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already install

In [0]:
import torch
import torchtext  # torchtextを使用
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')


In [0]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [0]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])



In [18]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())


4426
1475
1475


In [19]:
# datasetの中身を確認してみる
item = next(iter(dataset_train))
print(item.Text)
print("長さ：", len(item.Text))  # 長さを確認 [CLS]から始まり[SEP]で終わる。512より長いと後ろが切れる
print("ラベル：", item.Label)


tensor([    2,  8454,  6172,  1704,  5408,    16,  2109,    20,    16,    33,
           29,     6,   802, 28841, 28798,  2935,  8722,    40,   580,    26,
           20,    16,    33,  8454,     7,     9,     6,  3799,    18, 20093,
           14,    31,  4895,     8,   218,    11,  4021,    16,  6629,     5,
           28,     6,  8454,    11,  3062, 20206, 15591,     5,  1628,  6172,
          120,     8, 21966,     6,    36,  6740,     5,   745,   536, 11418,
           38,    13, 15414,   205,     8,  1216,  8454,  1178,  9494,     5,
           74,    35,  2154,  3111,    10,     5,     9,     6,  2154,    94,
        13493,   280,     7,    31,    36, 20498,  2154, 25239, 28578,  4815,
           38,     8,  1173, 28532, 28450,    91,   587,  8454,  2580,     5,
         1075,     9,     6,  2154,    12,  2857,    10,    13,     5,    45,
            8,  2381,    11,  5003,     6, 23429,   371,  9542,  1851,     6,
         1162,  1134,    29,     8,    26, 28468, 17985,  1851, 

In [20]:
# datasetの中身を文章に戻し、確認

print(tokenizer.convert_ids_to_tokens(item.Text.tolist()))  # 文章
dic_id2cat[int(item.Label)]  # id


['[CLS]', 'ビール', 'って', 'どう', 'やっ', 'て', '作ら', 'れ', 'て', 'いる', 'か', '、', 'ご', '##存', '##知', '?', '各社', 'から', '発売', 'さ', 'れ', 'て', 'いる', 'ビール', 'に', 'は', '、', 'さまざま', 'な', 'こだわり', 'が', 'ある', '模様', '。', 'それ', 'を', '知っ', 'て', 'おく', 'の', 'も', '、', 'ビール', 'を', '味', '##わう', '楽しみ', 'の', 'ひと', 'って', 'もの', '。', '早速', '、', '「', '大人', 'の', '社会', '科', '見学', '」', 'と', 'いこ', 'う', '。', '国内', 'ビール', '生産', '発祥', 'の', '地', '・', '横浜', '訪れ', 'た', 'の', 'は', '、', '横浜', '市', '鶴見', '区', 'に', 'ある', '「', 'キリン', '横浜', 'ビア', '##ビ', '##レッジ', '」', '。', 'じ', '##つ', '##は', '日本', 'における', 'ビール', '作り', 'の', '歴史', 'は', '、', '横浜', 'で', '始まっ', 'た', 'と', 'の', 'こと', '。', '伝統', 'を', '守る', '、', '由緒', '正', '##しき', '工場', '、', 'といった', 'ところ', 'か', '。', 'さ', '##っ', '##そく', '工場', 'の', '中', 'に', '脚', 'を', '踏み', '##入れる', 'と', '、', 'こんな', '泡', 'の', '演出', 'が', 'お', '出迎', '##え', '。', 'う', '##ぅ', '、', '自然', 'と', '喉', 'が', '鳴', '##っ', 'て', 'しまう', '。', 'さ', '##て', '、', 'ビール', 'の', '原料', 'は', '大きく', '分け', 'て', '3', 'つ', '。', '二条', '大', '##麦',

'livedoor-homme'

In [0]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}


In [22]:
# DataLoaderの動作確認 

batch = next(iter(dl_test))
print(batch)
print(batch.Text[0].shape)
print(batch.Label.shape)


[torchtext.data.batch.Batch of size 16]
	[.Text]:('[torch.LongTensor of size 16x512]', '[torch.LongTensor of size 16]')
	[.Label]:[torch.LongTensor of size 16]
torch.Size([16, 512])
torch.Size([16])


## 準備3：BERTのクラス分類用のモデルを用意する

Huggingfaceさんのをそのまま使うのではなく、BERTのbaseだけ使い、残りは自分で実装する

In [23]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
print(model)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [0]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        return output


In [25]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


ネットワーク設定完了


## 準備4：BERTのファインチューニングの設定

In [0]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True


In [0]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

## 5. 訓練を実施

In [0]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [29]:
# 学習・検証を実行する。1epochに2分ほどかかります
num_epochs = 4
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 2.1159 || 10iter. || 本イテレーションの正解率：0.25
イテレーション 20 || Loss: 1.9992 || 10iter. || 本イテレーションの正解率：0.375
イテレーション 30 || Loss: 1.5570 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 40 || Loss: 1.3301 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 50 || Loss: 1.7788 || 10iter. || 本イテレーションの正解率：0.3125
イテレーション 60 || Loss: 0.8538 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 70 || Loss: 0.9669 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 80 || Loss: 0.8314 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 90 || Loss: 0.4187 || 10iter. || 本イテレーションの正解率：1.0
イテレーション 100 || Loss: 1.3798 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 110 || Loss: 0.7275 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 120 || Loss: 0.3552 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 130 || Loss: 0.7527 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 140 || Loss: 0.7155 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 150 || Loss: 0.3241 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 160 || Loss: 0.3351 || 10iter. || 本イテレーションの正解率：0.9375
イ

## テストデータでの性能を確認

In [30]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dl_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dl_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_acc))


100%|██████████| 93/93 [00:24<00:00,  3.79it/s]

テストデータ1475個での正解率：0.9261


https://yoheikikuta.github.io/bert-japanese/

https://github.com/yoheikikuta/bert-japanese

の「BERT with SentencePiece for Japanese text.」

では、入力テキストにタイトルを含めていますが、今回はタイトルは除いています。

同様にタイトルを抜いている、[BERTを用いた日本語文書分類タスクの学習・ハイパーパラメータチューニングの実践例](https://medium.com/karakuri/bert%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%97%A5%E6%9C%AC%E8%AA%9E%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%82%BF%E3%82%B9%E3%82%AF%E3%81%AE%E5%AD%A6%E7%BF%92-%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%AE%E5%AE%9F%E8%B7%B5%E4%BE%8B-2fa5e4299b16)でも、正解率が92%ちょっととなっており、ほぼ同じ正解率が得られました。

以上。